In [1]:
import keras
import tensorflow as tf
from keras.layers import Dense,BatchNormalization
from keras.applications import DenseNet121
from keras import Model
from keras.preprocessing.image import ImageDataGenerator

In [2]:
import os

base_dir = 'real-vs-fake'
train_dir = os.path.join(base_dir,'train')
valid_dir = os.path.join(base_dir,'valid')
test_dir = os.path.join(base_dir,'test')

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True
)
valid_datagen = ImageDataGenerator(
    rescale=1./255
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    class_mode='binary',
    shuffle=True,
    target_size=(250,250),
    batch_size=64
)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    class_mode='binary',
    shuffle=True,
    target_size=(250,250),
    batch_size=64
)
test_generator = valid_datagen.flow_from_directory(
    test_dir,
    class_mode='binary',
    shuffle=True,
    target_size=(250,250),
    batch_size=64
)


Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [4]:
densenet = DenseNet121(weights='imagenet',include_top=False,input_shape=(250,250,3))
#densenet.summary()
for layer in densenet.layers:
    layer.trainable=False

29089792/29084464 [==============================] - 11s 0us/step


In [5]:
output = densenet.output
x = keras.layers.GlobalAveragePooling2D()(output)
x = Dense(512, activation=tf.nn.relu)(x)
x = BatchNormalization()(x)
x = keras.layers.Dropout(0.3)(x)
x = Dense(1, activation = 'sigmoid')(x)

model = Model(densenet.input,x)

model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.RMSprop(learning_rate=0.001), metrics=['accuracy'])

In [6]:
from keras.callbacks import CSVLogger, Callback, ReduceLROnPlateau

class basecall(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy')>0.98:
            print('/Stopped at 98%')
            self.model.stop_training = True
callback1 = CSVLogger('train.log')
callback2 = basecall()
callback3 = ReduceLROnPlateau(
    monitor='val_loss',
    factor = 0.3,
    verbose=1,
    patience=3
)

In [7]:
history = model.fit(
    train_generator,
    verbose=2,
    epochs=10,
    callbacks=[callback1,callback2,callback3],
    steps_per_epoch=100000/64,
    validation_data=valid_generator,
    validation_steps=20000/64
)

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
epochs = len(acc)
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Accuracy')
plt.figure()
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Loss')